<a href="https://colab.research.google.com/github/LIMDANBI/dacon_2021/blob/main/model_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler as MMS
from xgboost import XGBClassifier
from sklearn import tree

# 데이터 불러오기

In [11]:
train = pd.read_csv('/content/drive/MyDrive/dacon_2021/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon_2021/dataset/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/dacon_2021/dataset/sample_submission.csv')

In [12]:
train

,index,gender,Annual_income,income_type,Education,family_type,house_type,DAYS_BIRTH,working_day,FLAG_MOBIL,work_phone,phone,email,occyp_type,begin_month,car_reality,credit
0,0,F,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,-6.0,0,1.0
1,1,F,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,-5.0,1,1.0
2,2,M,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,-22.0,2,2.0
3,3,F,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,-37.0,1,0.0
4,4,F,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,-26.0,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,-2.0,0,1.0
26453,26453,F,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,-47.0,1,2.0
26454,26454,F,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,-25.0,1,2.0
26455,26455,M,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,-59.0,1,2.0


In [13]:
print(train.shape, test.shape)

(26457, 17) (10000, 16)


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   Annual_income  26457 non-null  float64
 3   income_type    26457 non-null  object 
 4   Education      26457 non-null  object 
 5   family_type    26457 non-null  object 
 6   house_type     26457 non-null  object 
 7   DAYS_BIRTH     26457 non-null  int64  
 8   working_day    26457 non-null  int64  
 9   FLAG_MOBIL     26457 non-null  int64  
 10  work_phone     26457 non-null  int64  
 11  phone          26457 non-null  int64  
 12  email          26457 non-null  int64  
 13  occyp_type     18286 non-null  object 
 14  begin_month    26457 non-null  float64
 15  car_reality    26457 non-null  int64  
 16  credit         26457 non-null  float64
dtypes: float64(3), int64(8), object(6)
memory usage: 3

# 랜덤 시드 고정

In [15]:
BASE_PATH = '.'

SEED = 10086
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

# 데이터 전처리

In [16]:
# 결측치 처리
train['occyp_type'] = train['occyp_type'].fillna('Null')
test['occyp_type'] = test['occyp_type'].fillna('Null')

In [17]:
# gender => 여성 - 0, 남성 - 1 
train['gender'] = train['gender'].replace({'F':0, 'M':1})
test['gender'] = test['gender'].replace({'F':0, 'M':1})

In [18]:
# FLAG_MOBIL: 전부 1 , index => 데이터 분석과 관련 x => 삭제 
train.drop('FLAG_MOBIL', axis=1, inplace=True)
test.drop('FLAG_MOBIL', axis=1, inplace=True)
del train['index']
del test['index']
print(train.shape, test.shape)

(26457, 15) (10000, 14)


In [19]:
# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [20]:
train_x = train.drop('credit', axis=1)
train_y = train[['credit']]
test_x = test
print(train_x.shape, train_y.shape, test_x.shape)

(26457, 49) (26457, 1) (10000, 49)


In [21]:
#수치형 데이터 전처리
train_x['DAYS_BIRTH'] = train_x['DAYS_BIRTH'] / train_x['DAYS_BIRTH'].min()
test_x['DAYS_BIRTH'] = test_x['DAYS_BIRTH'] / test_x['DAYS_BIRTH'].min()
train_x['working_day'] = train_x['working_day'] / train_x['working_day'].min()
test_x['working_day'] = test_x['working_day'] / test_x['working_day'].min()
train_x['begin_month'] = train_x['begin_month'] / train_x['begin_month'].min()
test_x['begin_month'] = test_x['begin_month'] / test_x['begin_month'].min()

train_x['Annual_income'] = train_x['Annual_income'] / train_x['Annual_income'].max()
test_x['Annual_income'] = test_x['Annual_income'] / test_x['Annual_income'].max()

In [22]:
train_x.head()

,gender,Annual_income,DAYS_BIRTH,working_day,work_phone,phone,email,begin_month,car_reality,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,Education_Academic degree,Education_Higher education,Education_Incomplete higher,Education_Lower secondary,Education_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Null,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,0.128571,0.552600,0.299688,0,0,0,0.100000,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0.157143,0.452449,0.098008,0,0,1,0.083333,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1,0.285714,0.758866,0.282187,0,1,0,0.366667,2,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0.128571,0.599873,0.133138,0,1,0,0.616667,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0.100000,0.597845,0.133966,0,0,0,0.433333,2,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [23]:
# train data와 validation data 분리
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y,
                                                  stratify=train_y,
                                                  test_size=0.2,
                                                  random_state = SEED)

# 모델 구현

## LGBMClassifier

In [25]:
model_LGB = LGBMClassifier(n_estimators=10000, num_leaves=50, subsample=0.8,learning_rate=0.01, #0.1
                      min_child_samples=60, max_depth=20)
evals = [(X_val, y_val)]
model_LGB.fit(X_train, y_train, early_stopping_rounds=100,
                 eval_set=evals, eval_metric='logloss',verbose=False)
pred_y = model_LGB.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), pred_y)}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.7411298195992897


## XGBClassifier

In [26]:
model_XGB = XGBClassifier(n_estimators=500, learning_rate =0.01, max_depth=20)
model_XGB.fit(X_train, y_train)
y_pred = model_XGB.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}") #0.7597714369509084

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.7633685352258209


## RandomForestClassifier

In [27]:
model_RF=RandomForestClassifier(n_estimators=500, max_features=8, random_state=SEED)
model_RF.fit(X_train, y_train)
y_pred = model_RF.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}") #0.7702669079823308

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


log_loss: 0.7857390325587111


## KNeighborsClassifier

In [28]:
model_KNN =  KNeighborsClassifier(n_neighbors=500, metric='minkowski', p=5, weights='uniform')
model_KNN.fit(X_train, y_train)
y_pred = model_KNN.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


log_loss: 0.8724149881460559


## LogisticRegression

In [29]:
model_LR = LogisticRegression()
model_LR.fit(X_train, y_train)
y_pred = model_LR.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.8652216513969185


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## DecisionTreeClassifier

In [30]:
model_TREE = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=SEED)
model_TREE.fit(X_train, y_train)
y_pred = model_TREE.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

log_loss: 0.8111744676790846


## MLPClassifier

In [31]:
model_MLP = MLPClassifier(hidden_layer_sizes = (100, 100), max_iter=200, activation='relu', alpha=3)
model_MLP.fit(X_train, y_train)
y_pred = model_MLP.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.8701166854743118


## VotingClassifier

In [34]:
model_VOTING = VotingClassifier(estimators=[('LGBM', model_LGB),
                                      ('XGB', model_XGB),
                                      ('RF', model_RF)],
                         voting='soft')
model_VOTING.fit(X_train, y_train)
pred_y = model_VOTING.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), pred_y)}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.744270608944694


In [35]:
model_VOTING2 = VotingClassifier(estimators=[('LGBM', model_LGB),
                                      ('XGB', model_XGB),
                                      ('RF', model_RF),
                                      ('TREE', model_TREE)],
                         voting='soft')
model_VOTING2.fit(X_train, y_train)
pred_y = model_VOTING2.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), pred_y)}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.720852765428186


In [36]:
model_VOTING3 = VotingClassifier(estimators=[('LGBM', model_LGB),
                                      ('XGB', model_XGB),
                                      ('RF', model_RF),
                                      ('MLP', model_MLP)],
                         voting='soft')
model_VOTING3.fit(X_train, y_train)
pred_y = model_VOTING3.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), pred_y)}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.7305115685043464


In [37]:
model_VOTING4 = VotingClassifier(estimators=[('LGBM', model_LGB),
                                      ('XGB', model_XGB)],
                         voting='soft')
model_VOTING4.fit(X_train, y_train)
pred_y = model_VOTING4.predict_proba(X_val)
print(f"log_loss: {log_loss(to_categorical(y_val['credit']), pred_y)}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


log_loss: 0.7572853529784471


# Test 

In [38]:
pred = model_VOTING2.predict_proba(test)

In [39]:
pred

array([[0.05226022, 0.20730385, 0.74043594],
       [0.36091177, 0.09601944, 0.54306879],
       [0.04897958, 0.14613325, 0.80488718],
       ...,
       [0.04161   , 0.19709089, 0.76129912],
       [0.23766393, 0.2771789 , 0.48515717],
       [0.08375417, 0.19181809, 0.72442773]])

In [40]:
submission.head()

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0


In [41]:
submission.loc[:,1:] = pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


In [42]:
submission

,index,0,1,2
0,26457,0.052260,0.207304,0.740436
1,26458,0.360912,0.096019,0.543069
2,26459,0.048980,0.146133,0.804887
3,26460,0.070819,0.119412,0.809768
4,26461,0.074708,0.217624,0.707669
...,...,...,...,...
9995,36452,0.138406,0.344172,0.517423
9996,36453,0.216942,0.401757,0.381300
9997,36454,0.041610,0.197091,0.761299
9998,36455,0.237664,0.277179,0.485157


In [43]:
submission.to_csv('/content/drive/MyDrive/dacon_2021/720852765428186.csv',index=False)